<a href="https://colab.research.google.com/github/khare19yash/Automated-Detection-of-Neuropsychiatric-Disorders/blob/master/asd_dfc_govinda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd gdrive/My Drive/IIITH_Internship/Neuro/ASD-DFC

/content/gdrive/My Drive/IIITH_Internship/Neuro/ASD-DFC


### Install libraries

In [3]:
!pip install nilearn

     |████████████████████████████████| 2.5MB 2.7MB/s 


In [4]:
!pip install wandb

     |████████████████████████████████| 1.8MB 2.8MB/s 
     |████████████████████████████████| 163kB 19.1MB/s 
     |████████████████████████████████| 133kB 16.3MB/s 
     |████████████████████████████████| 102kB 6.4MB/s 
     |████████████████████████████████| 102kB 8.4MB/s 
     |████████████████████████████████| 71kB 6.6MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=02bcab9c588cb8d5a193f3d3ca6eedd1ed61a54c61cc05e1812671ff3d84097d
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=898dc762b2a85a2924e3f1409f61296e9da33421334f7cb327bb3036d3d53866
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=725cdf5fcfbf2d5d7ac28c8cef20f9eee8ec0b56066c54

### Importing Libraries

In [5]:
#imports

import os
import numpy as np
import pandas as pd
import math
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
# from nilearn import datasets
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV
# from nilearn import connectome
import torch.nn.functional as F
# from transformers import BertTokenizer, BertModel
from sklearn.decomposition import PCA
from scipy.stats import ttest_rel, ttest_ind
import shutil
import random
import numpy.ma as ma
from docopt import docopt
from functools import partial
from sklearn import preprocessing
from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import wandb
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition
from sklearn import datasets
import warnings
import scipy
warnings.simplefilter("ignore", UserWarning)

!wandb login 93ff2b234de4902b71d7405831d980f0c3182650

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### Wandb

In [6]:
DEBUG = False
RESUME = False
SEED = 101

In [7]:

ID = wandb.util.generate_id() 
if RESUME:
    WANDB_ID = '162gtnyn'

wandb.init(project = 'asd3', name='ml-dfc', id='test')

config = wandb.config
config.epochs = 2 if DEBUG else 50           
config.bs =  4       # batch size
config.lr = 1e-4       # learning rateasd instalment
config.pat = 10         # patience for reduce lr on plateau
config.factor = 0.5    # factor by which to reduce lr
config.random_state = 101
config.window_size = 22
config.stride = 5
# config.in_channels = 19900
# config.out_channels = 32
# config.hidden_size = 768
config.hidden_dropout_prob = 0.3
config.n_layers = 1
config.heads = 2

wandb: Currently logged in as: pindi-krishna (use `wandb login --relogin` to force relogin)


### Data

In [8]:
folder_path = 'CC200'
data_dir = '../ASD/Datasets/ASD_Dataset/'
phenotype_file_path = '../ASD/Datasets/ASD_Dataset/Phenotypic_V1_0b_preprocessed1.csv'

### Helper Functions

In [9]:
#returns dataframe
def get_data(data_dir, folder_path, phenotype_file_path):
  file_paths = os.listdir(os.path.join(data_dir,folder_path))
  subject_ids = []
  for path in file_paths:
    index = path.find("5")
    subject_ids.append(int(path[index:index+5]))

  phenotype_df = pd.read_csv(phenotype_file_path)
  df = phenotype_df[phenotype_df['SUB_ID'].isin(subject_ids)]
  df = df[['SUB_ID','SITE_ID', 'DX_GROUP']]
  df.DX_GROUP = df.DX_GROUP.map({1: 1, 2:0})
  df['FILE_PATH'] = file_paths

  return df.values


def apply_threshold(data,data_dir,folder_path,threshold):
  n = data.shape[0]
  idx = []
  for i in range(n):
    path = os.path.join(data_dir,folder_path,data[i,3])
    timeseries_data = pd.read_csv(path, sep='\t')
    if timeseries_data.shape[0] > threshold:
        idx.append(i)
  return data[idx]

def confusion(g_turth,predictions):
    tn, fp, fn, tp = confusion_matrix(g_turth,predictions).ravel()
    accuracy = (tp+tn)/(tp+fp+tn+fn)
    sensitivity = (tp)/(tp+fn)
    specificty = (tn)/(tn+fp)
    return accuracy,sensitivity,specificty

#Utils
def gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))
        
def apply_pca(dfc_data,n_components=1024):
    n_sample,n_window,n_feat = dfc_data.shape
    dfc_data = dfc_data.reshape((n_sample*n_window,n_feat))
    pca = PCA(n_components=n_components)
    dfc_data = pca.fit_transform(dfc_data)
    dfc_data = dfc_data.reshape((n_sample,n_window,-1))
    return dfc_data

def get_regs(datas):

    """
    extracts the features as proposed in asd diagnet paper. 
    returns data with relevant features and also region indices
    """
    n_sample, n_window, n_feat = datas.shape
    datas = datas.reshape((n_sample*n_window,n_feat))
    avg=[]
    for ie in range(datas.shape[1]):
        avg.append(np.mean(datas[:,ie]))
    avg=np.array(avg)
    highs=avg.argsort()[-regnum:][::-1]
    lows=avg.argsort()[:regnum][::-1]
    regions=np.concatenate((highs,lows),axis=0)
    data = datas[regions]
    dfc_data = data.reshape((n_sample,n_window,-1))
    return dfc_data, regions


In [10]:
def compute_connectivity(functional):
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(functional))
        mask = np.invert(np.tri(corr.shape[0], k=-1, dtype=bool))
        m = ma.masked_where(mask == 1, mask)
        return ma.masked_where(m, corr).compressed()

def load_sfc(data_dir, X) : 
    sfc = []
    for filepath in X:
        df = pd.read_csv(os.path.join(data_dir, filepath), sep="\t", header=0)
        # df = df[:116]
        df = df.apply(lambda x: pd.to_numeric(x, errors='coerce'))
        ROIs = ["#" + str(y) for y in sorted([int(x[1:]) for x in df.keys().tolist()])]
        functional = np.nan_to_num(df[ROIs].to_numpy().T)
        functional = preprocessing.scale(functional, axis=1)
        n_rois, n_samples = functional.shape
        # n_slides = n_samples // window + 1
        fc = compute_connectivity(functional)
        fc = fc.astype(np.float32)
        sfc.append(fc)
    return np.stack(sfc, axis = 0)

def load_patient(data_dir, filepath, window=config.window_size,stride=config.stride, mode = 'tapered'):
    df = pd.read_csv(os.path.join(data_dir, filepath), sep="\t", header=0)
    df = df[:116]
    df = df.apply(lambda x: pd.to_numeric(x, errors='coerce'))
    ROIs = ["#" + str(y) for y in sorted([int(x[1:]) for x in df.keys().tolist()])]
    functional = np.nan_to_num(df[ROIs].to_numpy().T)
    functional = preprocessing.scale(functional, axis=1)
    n_rois, n_samples = functional.shape
    n_slides = n_samples // window + 1
    dfc_matrix = []
    size = (n_samples - window) + 1
    for i in range(0, size, stride):

        df = functional[:, i : i + window]
        if(mode == 'tapered') :
            win = np.ones((1, window))
            sigma_gauss = 3
            len_gauss = 7 * sigma_gauss + 1
            x = np.linspace(-len_gauss / 2, len_gauss / 2, len_gauss)
            gauss = np.exp(-x ** 2 / (2 * sigma_gauss ** 2))
            gauss = gauss / sum (gauss)
            # print(win.shape)
            # print(gauss.shape)
            gauss = gauss[np.newaxis, :]
            win = scipy.ndimage.correlate(win, gauss, mode = 'nearest')     # Important step
            win = np.tile(win, (n_rois, 1))    
            # print(win.shape)  
            df = win * df
        fc = compute_connectivity(df)
        fc = fc.astype(np.float32)
#         fc = fc[:512]
        dfc_matrix.append(fc)
    return np.stack(dfc_matrix,axis=0)

def load_dfc(data_dir, X, window = config.window_size,stride=config.stride, mode = 'tapered'):
    
    dfc = []
    for filepath in X:
        dfc.append(load_patient(data_dir, filepath, window=config.window_size,stride=config.stride, mode = mode))
    
    return np.stack(dfc,axis=0)

### Get Data

In [11]:
data = get_data(data_dir, folder_path, phenotype_file_path)
print(data.shape)       #tuple('SUB_ID','SITE_ID', 'DX_GROUP', 'FILE_PATH')

(1035, 4)


### Apply threshold on Time points for DFC

In [12]:
data = apply_threshold(data,data_dir,folder_path,100)
print(data.shape)       #tuple('SUB_ID','SITE_ID', 'DX_GROUP', 'FILE_PATH')

(1009, 4)


### To get Paths and labels

In [13]:
X = data[:, 3]      # Paths to inputs
Y = data[:, 2]      # labels
Y = Y.astype(np.float32)
print(X.shape)
print(Y.shape)
# del data

(1009,)
(1009,)


### Train-test-split

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 30, stratify = Y)
print(X_train.shape)
print(X_test.shape)

(807,)
(202,)


In [15]:
print(np.unique(Y_train, return_counts = True))
print(np.unique(Y_test, return_counts = True))

(array([0., 1.], dtype=float32), array([412, 395]))
(array([0., 1.], dtype=float32), array([103,  99]))


### DFC

In [16]:
data_dir = '../ASD/Datasets/ASD_Dataset/CC200'
fc_train = load_dfc(data_dir,X_train, mode = 'tapered')
fc_test = load_dfc(data_dir,X_test, mode = 'tapered')
print(fc_train.shape)
print(fc_test.shape)

(807, 19, 19900)
(202, 19, 19900)


In [17]:
num_sub, num_win = fc_train.shape[0], fc_train.shape[1]

In [18]:
fc_train_reshape = fc_train.reshape(np.prod(fc_train.shape[0 : 2]), fc_train.shape[2])        # check0
print(fc_train_reshape.shape)       # 807 * 19 = 15333

### Spectral clustering of DFC

In [19]:
# import numpy as np
import sys
from scipy.io import loadmat, savemat
from sklearn import mixture
from sklearn import manifold

num_comps = 12
cv_type = 'full'
num_init = 100
init_params = 'random'
X = fc_train_reshape.copy()

gmm_params = {}

# Spectral Embedding 

se = manifold.SpectralEmbedding(n_components=num_comps, affinity='rbf', gamma=0.0042, random_state=None, eigen_solver=None, n_jobs=-1)
X_new = se.fit_transform(X)                        # embedded wFCs   # a non-linear dimensionality reduction (num_train, )
gmm_params['embedding_data'] = X_new
gmm_params['affinity_matrix'] = se.affinity_matrix_ 

# Gaussian Mixture Modelling (GMM)

gmm = mixture.GaussianMixture(n_components = num_comps,
                              covariance_type = cv_type,
                              n_init = num_init,
                              max_iter = 100,
                              init_params = init_params,
                              verbose = 1)
gmm.fit(X_new)
gmm_params['soft_assigns'] = gmm.predict_proba(X_new) 
gmm_params['cluster_centers'] = gmm.means_
gmm_params['cluster_weights'] = gmm.weights_
gmm_params['cluster_covmats'] = gmm.covariances_

Initialization 0
  Iteration 10
Initialization converged: True
Initialization 1
  Iteration 10
Initialization converged: True
Initialization 2
  Iteration 10
  Iteration 20
Initialization converged: True
Initialization 3
  Iteration 10
  Iteration 20
  Iteration 30
Initialization converged: True
Initialization 4
  Iteration 10
  Iteration 20
  Iteration 30
Initialization converged: True
Initialization 5
  Iteration 10
  Iteration 20
  Iteration 30
Initialization converged: True
Initialization 6
  Iteration 10
Initialization converged: True
Initialization 7
  Iteration 10
  Iteration 20
Initialization converged: True
Initialization 8
  Iteration 10
  Iteration 20
Initialization converged: True
Initialization 9
  Iteration 10
Initialization converged: True
Initialization 10
  Iteration 10
Initialization converged: True
Initialization 11
  Iteration 10
  Iteration 20
Initialization converged: True
Initialization 12
  Iteration 10
Initialization converged: True
Initialization 13
  Iteratio

In [20]:
print(X_new.shape)

print(gmm_params['embedding_data'].shape)
print(gmm_params['affinity_matrix'].shape)

print(gmm_params['soft_assigns'].shape)
print(gmm_params['cluster_centers'].shape)
print(gmm_params['cluster_weights'].shape)
print(gmm_params['cluster_covmats'].shape)

(15333, 12)
(15333, 12)
(15333, 15333)
(15333, 12)
(12, 12)
(12,)
(12, 12, 12)


In [23]:
# np.save('GMM_X.npy', X_new)
import pickle
pickle.dump(gmm_params, open('./GMM_params.pkl', 'wb'))

In [24]:
num_comps = 12
num_train = len(X_train)

hard_assigns = np.argmax(gmm_params['soft_assigns'], axis = 1)          # why 2? and why max ?  # check1
hard_assigns_subj = np.reshape(hard_assigns, (num_train, num_win))
print('hard_assigns_subj shape : ', hard_assigns_subj.shape)

trans_mat = np.zeros((num_comps, num_comps))
for idx_subj in range(num_train):                                 # check2
    for idx_win in range(num_win - 1) :                           # check3
        
        from_state = hard_assigns_subj[idx_subj, idx_win]
        to_state = hard_assigns_subj[idx_subj, idx_win + 1]
        trans_mat[to_state, from_state] = trans_mat[to_state, from_state] + 1

print('trans_mat shape : ', trans_mat.shape)
print('trans_mat : ', trans_mat)

hard_assigns_subj shape :  (807, 19)
trans_mat shape :  (12, 12)
trans_mat :  [[1.2300e+02 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 2.0000e+00 0.0000e+00 2.0000e+00]
 [0.0000e+00 9.1400e+02 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 2.0000e+00 0.0000e+00]
 [0.0000e+00 2.0000e+00 4.6400e+02 0.0000e+00 0.0000e+00 1.0000e+00
  2.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 3.0000e+00]
 [3.0000e+00 0.0000e+00 0.0000e+00 6.3000e+01 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.0800e+02 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 3.0000e+00 2.0000e+00 0.0000e+00 0.0000e+00 1.0880e+03
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.6000e+01 0.0000e+00]
 [0.0000e+00 0.0000e+00 3.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00
  1.0700e+02 0.0000e+00 0.0000e+

In [25]:
# % normalize each column of transition matrix

col_sum = np.sum(trans_mat, axis = 1)              #check 4
print('column sum : ', col_sum)

ind = np.where(col_sum == 0)         # This should return the indices          # check5
print('Indices of zero : ', ind)

for i in ind :                           # check6
    trans_mat[:, i] = np.zeros((num_comps, 1))
    trans_mat[i, i] = 1

trans_mat_sum = np.sum(trans_mat, axis = 1)
trans_mat = trans_mat / trans_mat_sum
print('trans_mat sum  : ', trans_mat_sum)
print('trans_mat : ', trans_mat)

epss = 1e-5
for idx_comp in range(1, num_comps) :
    if(trans_mat[idx_comp, idx_comp] == 1) : 
        trans_mat[:, idx_comp] = trans_mat[:, idx_comp] + epss
        trans_mat[idx_comp, idx_comp] = trans_mat[idx_comp, idx_comp] - num_comps * epss            

column sum :  [  127.   916.   473.    67.   108.  1109.   111.   131.   130.    96.
 11081.   177.]
Indices of zero :  (array([], dtype=int64),)
trans_mat sum  :  [  127.   916.   473.    67.   108.  1109.   111.   131.   130.    96.
 11081.   177.]
trans_mat :  [[9.68503937e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 2.08333333e-02 0.00000000e+00 1.12994350e-02]
 [0.00000000e+00 9.97816594e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.80489126e-04 0.00000000e+00]
 [0.00000000e+00 2.18340611e-03 9.80972516e-01 0.00000000e+00
  0.00000000e+00 9.01713255e-04 1.80180180e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 9.02445628e-05 1.69491525e-02]
 [2.36220472e-02 0.00000000e+00 0.00000000e+00 9.40298507e-01
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  7.69230769e-03 0.00000000e+00 0.00000000e+00 0.00

In [29]:
ss_distrib, _ = np.linalg.eig(trans_mat)
# ss_distrib, _ = eigs(trans_mat, 1, 'largestabs')                     # check7
ss_distrib = ss_distrib / sum(ss_distrib)                            # check8
print(ss_distrib.shape)
print(ss_distrib)

(12,)
[0.08056837+7.93253732e-04j 0.08056837-7.93253732e-04j
 0.08140308+0.00000000e+00j 0.08310551+0.00000000e+00j
 0.08343875+0.00000000e+00j 0.08487132+0.00000000e+00j
 0.08463688+0.00000000e+00j 0.08406287+3.13242775e-05j
 0.08406287-3.13242775e-05j 0.084198  +0.00000000e+00j
 0.08422201+0.00000000e+00j 0.08486198+0.00000000e+00j]


## Ignore the below part

### ML Approach

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC


In [ ]:
svc = SVC()
skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=32)
parameters = {'kernel':('linear', 'rbf', 'linear', 'poly', 'rbf', 'sigmoid'), 'C':[0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}

clf = GridSearchCV(estimator = svc, param_grid = parameters, cv = skfold, return_train_score=True)
clf.fit(X, Y)

In [ ]:
print(clf.best_params_)

In [ ]:
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

### Data Loader

In [ ]:
class ASDDataset():
  def __init__(self,x,y,data_dir):
    self.x = x
    self.y = y
    self.data_dir = data_dir

  def __len__(self):
    return len(self.x)

  def __getitem__(self, item_idx):
#     filepath = self.x[item_idx]
#     x = torch.tensor(load_patient(self.data_dir,filepath),dtype=torch.float)
#     y = torch.tensor(self.y[item_idx],dtype=torch.float)

    return torch.tensor(self.x[item_idx],dtype=torch.float), torch.tensor(self.y[item_idx],dtype=torch.float)

### Train Test Functions

In [ ]:

def loss_fn(logits, targets):
  return nn.BCEWithLogitsLoss()(logits,targets)

def train_loop_fn(data_loader, model, optimizer, device, scheduler=None):
  model.train()
  train_loss = []
  final_targets = []
  final_predictions = []
  bar = tqdm(data_loader,leave=False)
  for (inputs, targets) in bar:
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()
    logits = model(inputs)
#     print(logits.shape)
#     print(targets.shape)
    loss = loss_fn(logits , targets)
    train_loss.append(loss.detach().cpu().numpy())
    
    proba = torch.sigmoid(logits).detach().cpu().numpy()
    predictions = np.ones_like(proba, dtype=np.int32)
    predictions[proba < 0.5] = 0
    final_targets.append(targets.detach().cpu().numpy())
    final_predictions.append(predictions)
    
    loss.backward()
    optimizer.step()

    if scheduler is not None:
      scheduler.step()

  final_targets = np.concatenate(final_targets)
  final_predictions = np.concatenate(final_predictions)
  train_accuracy = np.mean(final_targets == final_predictions)
  mlp_acc,mlp_sens,mlp_spef = confusion(final_targets,final_predictions)

    
  
  return np.mean(train_loss), mlp_acc

def eval_fn(data_loader, model, device):
    model.eval()
    eval_loss = []
    final_targets = []
    final_predictions = []
    bar = tqdm(data_loader,leave=False)
    
    with torch.no_grad():
      for (inputs, targets) in bar:
        inputs, targets = inputs.to(device), targets.to(device)
        logits = model(inputs)
        print(logits.shape)
        print(targets.shape)
        loss = loss_fn(logits , targets)
        eval_loss.append(loss.detach().cpu().numpy())
        
        proba = torch.sigmoid(logits).detach().cpu().numpy()
        predictions = np.ones_like(proba, dtype=np.int32)
        predictions[proba < 0.5] = 0
        
        final_predictions.append(predictions)
        final_targets.append(targets.detach().cpu().numpy())
        
    final_targets = np.concatenate(final_targets)
    final_predictions = np.concatenate(final_predictions)
    mlp_acc,mlp_sens,mlp_spef = confusion(final_targets,final_predictions)
    metrics_dict = {'accuracy': np.round(mlp_acc, 4), 
                        'senstivity' : np.round(mlp_sens,4), 
                        'specificity' : np.round(mlp_spef,4), 
                        'loss' : np.round(np.mean(eval_loss),4)}

    accuracy = np.mean(final_targets == final_predictions)
    eval_loss = np.mean(eval_loss)

    return accuracy, metrics_dict

def run_k_fold(X,Y,num_splits=10):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  data_dir = '../ASD/Datasets/ASD_Dataset/CC200'

  if num_splits is not None:
    # list to store metrics after each fold
    crossval_acc=[]
    crossval_sen=[]
    crossval_spec=[]
    crossval_loss=[]
    skf = StratifiedKFold(n_splits=num_splits, random_state=1, shuffle=True) 
    kk = 0
    for train_index, test_index in skf.split(X, Y):
        #init wandb 
        NAME = f'asd-cnnlstm-fold-{kk+1}'
        ID = f'cnnlstm-fold-{kk+1}'
#         wandb.init(project = 'asd1', name=NAME, id=ID)

        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]


        verbose = (True if (kk == 0) else False)

        train_dataset = ASDDataset(x_train,y_train, data_dir)
        test_dataset = ASDDataset(x_test,y_test, data_dir)

        train_dataloader = DataLoader(train_dataset,batch_size=config.bs,shuffle=True)
        test_dataloader = DataLoader(test_dataset,batch_size=config.bs,shuffle=False)

        """
        :param emb: Embedding dimension
        :param heads: nr. of attention heads
        :param depth: Number of transformer blocks
        :param seq_length: Expected maximum sequence length
        :param num_tokens: Number of tokens (usually words) in the vocabulary
        :param num_classes: Number of classes.
        :param max_pool: If true, use global max pooling in the last layer. If false, use global
                         average pooling.
        """

        model = CTransformer(1024, 8, 20, 4, 4, 1,device)  # (embedding dimension, #heads, depth, seq_length(4), num_tokens(4), num_classes)
        model.to(device)
#         wandb.watch(model, log='all')      # this logs everything in wandb account
        optimizer = optim.Adam(model.parameters(),lr=config.lr)

        for epoch in range(config.epochs):

            
            train_loss, train_accuracy = train_loop_fn(train_dataloader,model,optimizer,device)                                                   
            print(f'Epoch {epoch + 1}/{config.epochs}')

            wandb.log({'Train loss': train_loss,
                        'Train accuracy': train_accuracy})

            content = f'Classifier Train loss: {(train_loss):.4f}, Train Acc: {(train_accuracy):.4f}'
            print(content)

        accuracy, metrics_dict = eval_fn(test_dataloader, model, device)
        print("-----------------------------")
        print(f'Fold {kk+1}/{num_splits}')
        content = f'{metrics_dict}'
        print(content)
        print("-----------------------------")

        crossval_acc.append(metrics_dict['accuracy'])
        crossval_sen.append(metrics_dict['senstivity'])
        crossval_spec.append(metrics_dict['specificity'])
        crossval_loss.append(metrics_dict['loss'])

        #save the model after each fold
        recorder = {'optimizer': optimizer.state_dict(),
        'model': model.state_dict(),
        'fold' : kk+1}

        torch.save(recorder, f'./Models/{NAME}.pt')
        
        kk = kk + 1

    print("*********************************")    
    print(f'Average Value after 10 Folds------->')
    content = f'Accuracy: {np.round(np.mean(crossval_acc),4)}, Senstivity: {np.round(np.mean(crossval_sen),4)}, Specificity: {np.round(np.mean(crossval_spec),4)}, Loss: {np.round(np.mean(crossval_loss),4)}'
    print(content)
    print("*********************************") 

In [ ]:
run_k_fold(dfc,Y)

Epoch 1/50
Classifier Train loss: 0.8620, Train Acc: 0.4769


Epoch 2/50
Classifier Train loss: 0.7189, Train Acc: 0.5000


Epoch 3/50
Classifier Train loss: 0.7148, Train Acc: 0.5022


Epoch 4/50
Classifier Train loss: 0.7153, Train Acc: 0.4890


Epoch 5/50
Classifier Train loss: 0.7067, Train Acc: 0.5132


...

Epoch 27/50
Classifier Train loss: 0.6968, Train Acc: 0.4868


Epoch 28/50
Classifier Train loss: 0.6957, Train Acc: 0.4934


Epoch 29/50
Classifier Train loss: 0.6973, Train Acc: 0.5132


Epoch 30/50
Classifier Train loss: 0.6953, Train Acc: 0.4978


KeyboardInterrupt: ignored